In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
app_test = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/application_test.csv.zip', compression='zip')
app_train = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/application_train.csv.zip', compression='zip')

# preprocessing 

In [6]:
print(app_test.shape, app_train.shape)

(48744, 121) (307511, 122)


In [7]:
df = app_train.append(app_test).reset_index()

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in df:
    if df[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(df[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            df[col] = le.transform(df[col])            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [10]:
# one hot encoding
categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
df = pd.get_dummies(df, columns= categorical_columns, dummy_na= True)
print(df.dtypes.value_counts())

uint8      147
float64     66
int64       44
dtype: int64


In [11]:
#df.isnull().sum().sort_values(ascending = False).head()

In [12]:
df_train = df[df['TARGET'].notnull()]
y = df_train['TARGET']
df_test = df[df['TARGET'].isnull()]

In [13]:
del df_train['TARGET']
del df_test['TARGET']

In [14]:
print(df_train.shape, df_test.shape)

(307511, 256) (48744, 256)


# adding features from other datasets

In [15]:
bureau = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/bureau.csv.zip', compression='zip')
bureau_bal = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/bureau_balance.csv.zip', compression='zip')

# cross validation

In [10]:
#from sklearn.model_selection import StratifiedKFold

#skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=27)

#X_train, X_test, y_train, y_test = skf.split(X, y)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=42)